## Logistic Version of the Isolet Model Test in Wallaroo

The following demonstrates how to use a converted sk-learn Logistics ML model converted to ONNX  in Wallaroo.  This is meant to be a follow up to the tutorial `convert-sklearn-logistic-to-onnx.ipyb`, which demonstrates how to convert the sk-learn logicistics model to ONNX format and generates the file `isolet_logistic_model_numclass.onnx` that is used in this tutorial.

For more details on using Wallaroo workspaces, pipelines, and models, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

This tutorial includes the following assets:

* `isolet_test_data.tsv`:  A test file that can be used to verify the output of the converted logistic model.
* `test-converted-sklearn-logistics-to-onnx.ipynb`: This Jupyter Notebook that demonstrates the use of the converted sk-learn logistic ML model in ONNX with Wallaroo.

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame

## Test Steps

### Load Libraries

The first step in the process is to load the necessary libraries.

In [1]:
# from their artifacts

import wallaroo

from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Connect to Workspace

Either create or use an existing workspace.  In this example, we will connect to the existing workspace `isolettest` by using the `set_current_workspace` command.  For more details on connecting to a workspace, see the [Wallaroo SDK Essentials guide](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/).

![Wallaroo Workspace Example](./images/wallaroo-tutorials/isolet-workspace-example.png)

In [3]:
workspace_name = 'isolettest'
pipeline_name = 'isoletpipeline'
model_name = 'isolettest'

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

isoletpipeline = get_pipeline(pipeline_name)
isoletpipeline

name,isoletpipeline
created,2023-05-17 21:17:33.444632+00:00
last_updated,2023-05-17 21:17:33.444632+00:00
deployed,(none)
tags,
versions,145b768e-79f2-44fd-ab6b-14d675501b83
steps,


### Upload Model

Upload the `isolet_logistic_model_numclass.onnx` model as `isolet-test` with the `upload_model` method:

In [5]:
model = wl.upload_model(model_name, './isolet_logistic_model_numclass.onnx').configure()

In [6]:
isoletpipeline.add_model_step(model)

name,isoletpipeline
created,2023-05-17 21:17:33.444632+00:00
last_updated,2023-05-17 21:17:33.444632+00:00
deployed,(none)
tags,
versions,145b768e-79f2-44fd-ab6b-14d675501b83
steps,


### Deploy Pipeline

Pipelines can be deployed either with the Wallaroo Dashboard user interface or through the SDK.  For this example, we have the pipeline `isolet-pipeline` deployed and assigned to the variable `isoletpipeline`.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

![Wallaroo Workspace Example](./images/wallaroo-tutorials/isolet-pipeline-example.png)

We can check the status of the pipeline with the Wallaroo pipeline `status` method.

In [7]:
isoletpipeline.deploy()

name,isoletpipeline
created,2023-05-17 21:17:33.444632+00:00
last_updated,2023-05-17 21:17:44.618177+00:00
deployed,True
tags,
versions,"c129b33c-cefc-4873-ad2c-d186fe2b8228, 145b768e-79f2-44fd-ab6b-14d675501b83"
steps,isolettest


### Load the Data

We'll load the data from the file `isolet_test_data.tsv`.

In [8]:
# read in data
testdata = pd.read_csv('isolet_test_data.tsv', sep='\t')

We'll display the data shape to verify it has 9 rows and 617 columsn per row.

In [9]:
# report the size of the data  -  9 rows, 617 columns each
testdata.shape

(9, 617)

### Run Inference

Use the Wallaroo pipeline `infer` method to run an inference and store the results into the variable `result`, then display the data from the result.

In [10]:
# convert to a tensor dataframe
inputdata = pd.DataFrame(columns=['tensor'])
for index, row in testdata.iterrows():
    inputdata = inputdata.append({'tensor': row.values.tolist()}, ignore_index=True)
result = isoletpipeline.infer(inputdata)
display(result['out.probabilities'])

0     [0.999963, 3.7014484e-05]
1     [0.99774987, 0.002250135]
2      [0.004735887, 0.9952641]
3      [0.031078666, 0.9689213]
4    [0.9996307, 0.00036931038]
5     [0.9992027, 0.0007972717]
6       [0.14781258, 0.8521874]
7     [0.080456555, 0.91954345]
8     [0.9987874, 0.0012125671]
Name: out.probabilities, dtype: object

### Closing

Once the tests are done, the pipeline should be undeployed to allocate the resources back to the Wallaroo instance.  For more details, see the [Wallaroo Pipeline Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-pipeline-management/).

In [11]:
isoletpipeline.undeploy()

name,isoletpipeline
created,2023-05-17 21:17:33.444632+00:00
last_updated,2023-05-17 21:17:44.618177+00:00
deployed,False
tags,
versions,"c129b33c-cefc-4873-ad2c-d186fe2b8228, 145b768e-79f2-44fd-ab6b-14d675501b83"
steps,isolettest
